In [1]:
%pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 9.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from gensim.models.ldamodel import LdaModel

# Load the LDA model
loaded_lda_model = LdaModel.load('/content/drive/MyDrive/Colab Notebooks/alex/models/lda_15_topics/lda_model_15_topics.gensim')

# Now you can access information from the loaded model.
# For example, you can print the topics:
print("Topics:")
for topic_id, topic in loaded_lda_model.print_topics():
    print(f"Topic {topic_id}: {topic}")

# You can also access the dictionary (id2word)
print("\nDictionary:")
print(loaded_lda_model.id2word)

# And the number of topics
print("\nNumber of topics:")
print(loaded_lda_model.num_topics)

Topics:
Topic 0: 0.040*"state" + 0.018*"game" + 0.018*"michigan" + 0.017*"big" + 0.016*"texas" + 0.014*"college" + 0.014*"team" + 0.014*"season" + 0.013*"florida" + 0.013*"alabama"
Topic 1: 0.023*"school" + 0.018*"year" + 0.017*"team" + 0.016*"coach" + 0.015*"high" + 0.014*"season" + 0.014*"college" + 0.013*"player" + 0.012*"woman" + 0.011*"basketball"
Topic 2: 0.032*"gopher" + 0.024*"hockey" + 0.023*"goal" + 0.021*"minnesota" + 0.021*"game" + 0.013*"team" + 0.012*"season" + 0.009*"first" + 0.009*"boston" + 0.008*"four"
Topic 3: 0.013*"sport" + 0.007*"game" + 0.007*"fan" + 0.006*"basketball" + 0.006*"new" + 0.005*"one" + 0.005*"nil" + 0.005*"people" + 0.004*"time" + 0.004*"day"
Topic 4: 0.015*"said" + 0.014*"ncaa" + 0.014*"sport" + 0.014*"college" + 0.013*"school" + 0.012*"athlete" + 0.008*"state" + 0.008*"university" + 0.008*"football" + 0.007*"would"
Topic 5: 0.023*"woman" + 0.016*"u" + 0.015*"championship" + 0.013*"world" + 0.011*"olympic" + 0.011*"first" + 0.011*"2023" + 0.008*"uni

In [3]:
from gensim import corpora

# Load the corpus from the .mm file
corpus = corpora.MmCorpus('/content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/corpus.mm')

print("Corpus loaded successfully.")
# You can optionally print the corpus to inspect its structure
# print(list(corpus))

Corpus loaded successfully.


In [4]:
import pandas as pd

# Load the tokenized data from the Excel file
try:
    tokenized_data_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/alex/updated_ tokenized_newspaper_data.xlsx')
    print("Excel file loaded successfully.")
    # Assuming the original text is in a column named 'text' or similar.
    # Please adjust the column name if it's different in your Excel file.
    if 'Main_text_tokenized' in tokenized_data_df.columns:
        texts = tokenized_data_df['Main_text_tokenized'].tolist()
        print(f"Extracted {len(texts)} documents.")
    else:
        texts = None
        print("Could not find a 'text' column in the Excel file. Please make sure the column containing original texts is named 'text' or update the code.")

except FileNotFoundError:
    print("Error: The file was not found at the specified path.")
except Exception as e:
    print(f"An error occurred while reading the Excel file: {e}")

Excel file loaded successfully.
Extracted 149703 documents.


In [5]:
#TFinding the dominant topic in each sentence

def format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts): # Updated to use loaded_lda_model and corpus, and texts=None
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = pd.concat([sent_topics_df, pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]).to_frame().T], ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

     # Add original text to the end of the output
    if texts is not None: # Added check if texts is not None
        contents = pd.Series(texts)
        sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


# Assuming texts is available as data_words or you provide another list/Series of texts
# df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts) # Updated to use loaded_lda_model and corpus, commented out as data_words is undefined
# To run this cell, define `data_words` with your text data before calling the function.
# For example:
# data_words = ["your document text 1", "your document text 2", ...]
# Then uncomment the line below:
df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts)

# Format
if 'df_topic_sents_keywords' in locals(): # Added check if df_topic_sents_keywords is defined
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
    # Show
    display(df_dominant_topic.head(20)) # Use display for better formatting

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,10,0.4354,"said, like, year, player, coach, going, would,...","['14', 'year', 'coaching', '39', 'year', 'livi..."
1,1,10,0.2624,"said, like, year, player, coach, going, would,...","['see', 'sidebar', '02c', 'ruston', 'la', 'din..."
2,2,10,0.3697,"said, like, year, player, coach, going, would,...","['gave', 'garden', 'exactly', 'missed', 'lonel..."
3,3,10,0.3112,"said, like, year, player, coach, going, would,...","['indianapolis', 'ten', 'year', 'ago', 'chuck'..."
4,4,12,0.3408,"season, draft, player, nba, pick, coach, baske...","['robinson', 'talk', 'ranger', 'stanley', 'cup..."
5,5,7,0.3633,"baseball, game, run, ppg, series, inning, soft...","['usc', 'conference', 'pacific10', 'coach', 'm..."
6,6,12,0.5554,"season, draft, player, nba, pick, coach, baske...","['ranger', 'signed', 'center', 'michael', 'nyl..."
7,7,12,0.487,"season, draft, player, nba, pick, coach, baske...","['gavitt', 'five', 'others', 'elected', 'hall'..."
8,8,10,0.3186,"said, like, year, player, coach, going, would,...","['almost', 'mythical', 'element', 'lsu', 'seni..."
9,9,10,0.3069,"said, like, year, player, coach, going, would,...","['ncaa', 'woman', 'jim', 'izard', 'diana', 'vi..."


In [6]:
# Display the first few rows of the dataframe to see the structure
display(df_dominant_topic.head())

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,10,0.4354,"said, like, year, player, coach, going, would,...","['14', 'year', 'coaching', '39', 'year', 'livi..."
1,1,10,0.2624,"said, like, year, player, coach, going, would,...","['see', 'sidebar', '02c', 'ruston', 'la', 'din..."
2,2,10,0.3697,"said, like, year, player, coach, going, would,...","['gave', 'garden', 'exactly', 'missed', 'lonel..."
3,3,10,0.3112,"said, like, year, player, coach, going, would,...","['indianapolis', 'ten', 'year', 'ago', 'chuck'..."
4,4,12,0.3408,"season, draft, player, nba, pick, coach, baske...","['robinson', 'talk', 'ranger', 'stanley', 'cup..."


In [7]:
# Sort documents within each topic by percentage contribution in descending order
sorted_documents_by_topic = df_dominant_topic.sort_values(by=['Dominant_Topic', 'Topic_Perc_Contrib'], ascending=[True, False])

# Display the sorted document indexes for each topic
print("Document Indexes for Each Topic (Sorted by Percentage Contribution):")
for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    print(f"\nTopic {topic_num}:")
    # Get documents for the current topic and select only the 'Document_No' column
    topic_documents = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num]['Document_No']
    # Display the document indexes (first 20 as an example)
    print(topic_documents.tolist()[:20])

Document Indexes for Each Topic (Sorted by Percentage Contribution):

Topic 0:
[144042, 14464, 138838, 144043, 143977, 123595, 123346, 124951, 5019, 141418, 115297, 132891, 123345, 115294, 138375, 140980, 123347, 140979, 125074, 127493]

Topic 1:
[3163, 66229, 64700, 3155, 70296, 71089, 17560, 83855, 34546, 105769, 6914, 68576, 76467, 2919, 23432, 51516, 57636, 136344, 45644, 49396]

Topic 2:
[5037, 137523, 22572, 21692, 2563, 14607, 12479, 4732, 5021, 95156, 83747, 24103, 137533, 142444, 99549, 121170, 137336, 138598, 137455, 129260]

Topic 3:
[38825, 144759, 60071, 24353, 24572, 49210, 122316, 147347, 127351, 65611, 139919, 72564, 80583, 72954, 129930, 64275, 19278, 137576, 110460, 45590]

Topic 4:
[69832, 70792, 14390, 32177, 124934, 146373, 127699, 132924, 70785, 112461, 1145, 138582, 148124, 148125, 105593, 143165, 143166, 92629, 24195, 43289]

Topic 5:
[60336, 21267, 60341, 24696, 31532, 24580, 13955, 27528, 105809, 54726, 144713, 37076, 103299, 19378, 53263, 122244, 12611, 74149

In [8]:
# Get the total number of documents for each dominant topic
topic_counts = df_dominant_topic['Dominant_Topic'].value_counts().sort_index()

print("Total number of documents for each topic:")
display(topic_counts)

Total number of documents for each topic:


,count
Dominant_Topic,
0,5590
1,10649
2,3966
3,12109
4,15375
5,9731
6,18873
7,2370
8,1121


In [9]:
# Get the top 100 documents for each topic
top_100_documents_per_topic = pd.DataFrame()

for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    # Get the top 100 documents for the current topic
    top_100 = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num].head(100)
    # Append to the results DataFrame
    top_100_documents_per_topic = pd.concat([top_100_documents_per_topic, top_100], ignore_index=True)

# Display the top 100 documents for each topic
print("Top 100 Documents for Each Topic (Sorted by Percentage Contribution):")

# Iterate through topics and display the top documents for each
for topic_num in top_100_documents_per_topic['Dominant_Topic'].unique():
    print(f"\n--- Topic {topic_num} ---")
    display(top_100_documents_per_topic[top_100_documents_per_topic['Dominant_Topic'] == topic_num])

Top 100 Documents for Each Topic (Sorted by Percentage Contribution):

--- Topic 0 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,144042,0,0.9402,"state, game, michigan, big, texas, college, te...","['vaccination', 'reprinted', 'new', 'york', 't..."
1,14464,0,0.9378,"state, game, michigan, big, texas, college, te...","['one', 'thing', 'sure', 'day', 'beach', 'wedn..."
2,138838,0,0.9326,"state, game, michigan, big, texas, college, te...","['columbia', 'sc', 'south', 'carolina', 'quart..."
3,144043,0,0.9272,"state, game, michigan, big, texas, college, te...","['chicago', 'red', 'star', 'bringing', 'new', ..."
4,143977,0,0.8961,"state, game, michigan, big, texas, college, te...","['ncaa', 'baseball', 'tournament', 'remain', '..."
...,...,...,...,...,...
95,133818,0,0.7872,"state, game, michigan, big, texas, college, te...","['tableau', 'riot', 'police', 'screaming', 'cr..."
96,141752,0,0.7868,"state, game, michigan, big, texas, college, te...","['head', 'coach', 'deion', 'sander', 'making',..."
97,135037,0,0.7842,"state, game, michigan, big, texas, college, te...","['spring', 'day', 'lia', 'thomas', 'made', 'hi..."
98,132670,0,0.7841,"state, game, michigan, big, texas, college, te...","['newark', 'inevitable', 'first', 'time', 'jim..."



--- Topic 1 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
100,3163,1,0.9872,"school, year, team, coach, high, season, colle...","['wrapped', 'regular', 'season', 'second', 'ma..."
101,66229,1,0.9767,"school, year, team, coach, high, season, colle...","['pasadena', 'ucla', 'deserves', 'play', 'bowl..."
102,64700,1,0.9622,"school, year, team, coach, high, season, colle...","['one', 'day', 'nick', 'pita', 'return', 'wind..."
103,3155,1,0.941,"school, year, team, coach, high, season, colle...","['college', 'roundup', 'sophomore', 'jeff', 'h..."
104,70296,1,0.9397,"school, year, team, coach, high, season, colle...","['next', 'month', 'highcaliber', 'underclassma..."
...,...,...,...,...,...
195,20319,1,0.7503,"school, year, team, coach, high, season, colle...","['start', 'march', 'madness', 'always', 'bring..."
196,117925,1,0.7503,"school, year, team, coach, high, season, colle...","['spark', 'sawdust', 'norm', 'marketing', 'fir..."
197,76878,1,0.7489,"school, year, team, coach, high, season, colle...","['everybody', 'watching', 'louisville', 'put',..."
198,98406,1,0.7485,"school, year, team, coach, high, season, colle...","['one', 'moment', 'ucla', 'basketball', 'team'..."



--- Topic 2 ---


...

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
1200,90124,12,0.8763,"season, draft, player, nba, pick, coach, baske...","['illinois', 'hoped', 'ahmad', 'starks', 'woul..."
1201,126620,12,0.8735,"season, draft, player, nba, pick, coach, baske...","['southern', 'california', 'quarterback', 'jt'..."
1202,90733,12,0.8599,"season, draft, player, nba, pick, coach, baske...","['arlington', 'tex', 'arlington', 'tex', 'mich..."
1203,5502,12,0.8087,"season, draft, player, nba, pick, coach, baske...","['rsec', 'state', 'tournament', 'time', 'seems..."
1204,97330,12,0.8049,"season, draft, player, nba, pick, coach, baske...","['washington', 'wizard', 'relatively', 'muted'..."
...,...,...,...,...,...
1295,106543,12,0.6825,"season, draft, player, nba, pick, coach, baske...","['oregon', 'wide', 'receiver', 'olympic', 'hur..."
1296,8407,12,0.6818,"season, draft, player, nba, pick, coach, baske...","['long', 'beach', 'state', 'expected', 'select..."
1297,120239,12,0.6818,"season, draft, player, nba, pick, coach, baske...","['editor', 'note', 'story', 'updated', 'latest..."
1298,138405,12,0.6793,"season, draft, player, nba, pick, coach, baske...","['ucla', 'come', 'give', 'good', 'try', 'best'..."



--- Topic 13 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
1300,83806,13,0.9788,"point, state, iowa, scored, 12, win, first, 16...","['state', 'college', 'pa', 'penn', 'state', 'f..."
1301,19155,13,0.9358,"point, state, iowa, scored, 12, win, first, 16...","['anaheim', 'took', 'turn', 'assessing', 'poss..."
1302,94889,13,0.9333,"point, state, iowa, scored, 12, win, first, 16...","['sweet', '16', 'schedule', 'today', 'game', '..."
1303,45451,13,0.9285,"point, state, iowa, scored, 12, win, first, 16...","['freshman', 'casey', 'nogueira', 'goal', 'ass..."
1304,44483,13,0.9144,"point, state, iowa, scored, 12, win, first, 16...","['underrated', 'part', 'ncaa', 'tournament', '..."
...,...,...,...,...,...
1395,13798,13,0.8134,"point, state, iowa, scored, 12, win, first, 16...","['sophomore', 'dave', 'kirschner', 'set', 'lyc..."
1396,147189,13,0.813,"point, state, iowa, scored, 12, win, first, 16...","['5', 'florida', 'state', 'softball', 'advance..."
1397,32033,13,0.8129,"point, state, iowa, scored, 12, win, first, 16...","['three', 'year', 'ago', 'southern', 'californ..."
1398,13797,13,0.812,"point, state, iowa, scored, 12, win, first, 16...","['iona', 'led', 'richard', 'kiplagat', 'finish..."



--- Topic 14 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
1400,58040,14,0.9778,"tournament, team, ncaa, basketball, four, game...","['scottie', 'reynolds', 'scored', '22', 'point..."
1401,59112,14,0.9761,"tournament, team, ncaa, basketball, four, game...","['tyrus', 'thomas', 'glen', 'big', 'baby', 'da..."
1402,10637,14,0.9553,"tournament, team, ncaa, basketball, four, game...","['jake', 'serfass', 'crossroad', 'sort', 'earl..."
1403,71163,14,0.9513,"tournament, team, ncaa, basketball, four, game...","['bruin', 'notebook', 'wilmington', 'bruin', '..."
1404,9378,14,0.9509,"tournament, team, ncaa, basketball, four, game...","['unheardof', 'nfl', 'draft', '20yearolds', 'b..."
...,...,...,...,...,...
1495,24477,14,0.7803,"tournament, team, ncaa, basketball, four, game...","['hofstra', 'lost', 'topseeded', 'stanford', '..."
1496,99928,14,0.78,"tournament, team, ncaa, basketball, four, game...","['opening', 'loss', 'sacramento', 'state', 'ar..."
1497,81414,14,0.7792,"tournament, team, ncaa, basketball, four, game...","['matt', 'youmans', 'la', 'vega', 'reviewjourn..."
1498,31986,14,0.7789,"tournament, team, ncaa, basketball, four, game...","['special', 'section', 'ncaa', 'tournament', '..."


In [10]:
# Define the path to save the Excel file in your Google Drive
excel_file_path = '/content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_15_topics.xlsx'

# Save the DataFrame to an Excel file
try:
    top_100_documents_per_topic.to_excel(excel_file_path, index=False)
    print(f"Successfully saved the top 100 documents per topic to: {excel_file_path}")
except Exception as e:
    print(f"An error occurred while saving the Excel file: {e}")

Successfully saved the top 100 documents per topic to: /content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_15_topics.xlsx


##### **© 2024–2025 MD Rafsun Sheikh**
##### **Licensed under the Apache License, Version 2.0.**